# Result analysis
- Do inference
- Try to visualize the knowledge after learning from data

In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
import random
import numpy as np
import pprint
from time import gmtime, strftime
import glob
import torch
import torch.nn as nn
# from torch.backends import cudnn

from Logging import Logger
# from config import cfg
from model import MnistModel
from evaluator import Evaluator
from pgd_attack import LinfPGDAttack
from utils import *

In [65]:
from types import SimpleNamespace


cfg = SimpleNamespace(
    seed=0,
    randomize=False,
    log_dir='logs',
    exp_name='debug',
    no_log=False,
    data_dir='data/mnist',
    batch_size=200,
    eval_batch_size=200,
    eval_model="snapshots/debug/checkpoint_100.pth",
    epsilon=0.3,
    k=40,
    alpha=0.01,
    random_start=False,
)

In [28]:
import random
# cudnn.benchmark = True
# os.environ["CUDA_VISIBLE_DEVICES"] = cfg.gpus
if not cfg.randomize:
    # set fixed seed
    random.seed(cfg.seed)
    np.random.seed(cfg.seed)
    torch.manual_seed(cfg.seed)
    torch.cuda.manual_seed(cfg.seed)
    
log_path = os.path.join(cfg.log_dir, cfg.exp_name)
mkdir_if_missing(log_path)

# if not cfg.no_log:
#     log_name = cfg.exp_name + "_eval_log_" + \
#             strftime("%Y-%m-%d_%H-%M-%S", gmtime()) + '.txt'
#     sys.stdout = Logger(os.path.join(log_path, log_name))

In [41]:
print("Input Args: ")
pprint.pprint(cfg)

In [42]:
_, test_loader, num_class, img_size = get_data_loader(
    data_dir=cfg.data_dir, 
    batch_size=cfg.batch_size, 
    test_batch_size=cfg.eval_batch_size, 
    num_workers=4
)
print("Num classes: ", num_class)
print("Img size: ", img_size)
print("Num test batches: ", len(test_loader))

In [44]:
model = MnistModel()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

is_cuda = False
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
if torch.cuda.is_available():
    model = model.cuda()
    is_cuda = True

In [46]:
if cfg.eval_model is None:
    print('evaluate model is not specified')
    snapshots = []
elif os.path.isdir(cfg.eval_model):
    snapshots = glob.glob(os.path.join(cfg.eval_model, '*.pth'))
    snapshots = sorted(snapshots, key=lambda x: 
        int(os.path.basename(x).split('_')[1].split('.')[0]))
elif os.path.isfile(cfg.eval_model):
    snapshots = [cfg.eval_model]
else:
    snapshots = []

In [71]:
def load_model2(model, path):
    if not os.path.isfile(path):
        raise IOError('model: {} is non-exists'.format(path))
    if hasattr(model, 'module'):
        module = model.module
    else:
        module = model
    checkpoint = torch.load(path, map_location=device)
    if 'state_dict' in checkpoint:
        state_dict = checkpoint['state_dict']
    else:
        state_dict = checkpoint
    module.load_state_dict(state_dict, strict=False)
    print('Params Loaded from: {}'.format(path))

In [66]:
for snapshot in snapshots:
    load_model_cpu(model, snapshot)
    attack = LinfPGDAttack(model=model, epsilon=cfg.epsilon, k=cfg.k, 
                        alpha=cfg.alpha, random_start=cfg.random_start)
    evaluator = Evaluator(model=model, attack=attack, is_cuda=is_cuda, verbose=False)
    acc, adv_acc = evaluator.evaluate(test_loader)
    print("natural     accuracy: {:.4f}".format(acc))
    print("adversarial accuracy: {:.4f}".format(adv_acc))

In [67]:
acc

0.9865

In [68]:
adv_acc

0.9227

In [70]:
device

NameError: name 'device' is not defined